In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/Students/Negin/

/content/drive/My Drive/Colab Notebooks/Students/Negin


In [28]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics.cluster import adjusted_rand_score as ARI
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
import torch.nn.functional as F

import warnings
warnings.filterwarnings('ignore')
from util import eval, loadDataset, normalization, kmeansInitialization

eps = torch.tensor(10 ** -10)





datasets = ['3Sources.npy',
'BBCSport.npy',
'Caltech101.npy',
'Caltech_2.npy',
'Citeseer.npy',
'Coil100.npy',
'Cora.npy',
'EYaleB10.npy',
'Handwritten.npy',
'MNIST10.npy',
'UCIdigit.npy',
'Umist.npy',
'Yale32.npy',
'Yeast.npy',
'Cora2.npy',
'texas.npy',
'wisconsin.npy',
'washington.npy',
'cornell.npy']


X0, Y, vv, c = loadDataset(10)


r = 40
k = 5

# rr = 2

p = 2
gamma = -2
lamda = 0
mu = 10
beta1 = 1
beta2 = 1


iter = 200

U = []
V = []
X = []

# O = torch.ones(r, r)

for v in range(vv):


    X.append(torch.tensor(X0[v]).type(torch.float32))

    d, n = X[v].shape

    U.append(torch.rand(d, r))
    V.append(torch.rand(n, r))

Vs = torch.rand(n, r)
alpha = torch.ones(vv)/vv

G = torch.zeros(vv)


Wn = []
Wd = []
Dn = []
Dd = []
D =  []

for v in range(vv):

    An0 = kneighbors_graph(X[v].T, k, mode='connectivity', include_self=False).toarray()
    An0 = torch.tensor(An0).type(torch.float32)
    An0 = torch.maximum(An0, An0.T)

    Wn.append(An0)

    Dn0 = torch.diag(torch.sum(An0, dim = 1))
    Dn.append(Dn0)


    Ad0 = kneighbors_graph(X[v], k, mode='connectivity', include_self=False).toarray()
    Ad0 = torch.tensor(Ad0).type(torch.float32)
    Ad0 = torch.maximum(Ad0, Ad0.T)

    Wd.append(Ad0)

    Dd0 = torch.diag(torch.sum(Ad0, dim = 1))
    Dd.append(Dd0)

    D.append(torch.zeros(n,n))

# Optimization
err = torch.zeros(iter)

for t in range(iter):

    for v in range(vv):

      # Updating Gm
      G[v] = (torch.sum(torch.norm(X[v] - U[v] @ V[v].T, dim=0)** (p/2))) ** (1/p-1)

      # Updating Dm
      D[v] = torch.diag(torch.norm(X[v] - U[v] @ V[v].T, dim=0)** (p/2 - 1))

      # Updating Um
      Un = alpha[v] * G[v] * (X[v]          @ D[v] @ V[v]) + beta1 * (Wd[v] @ U[v])
      Ud = alpha[v] * G[v] * (U[v] @ V[v].T @ D[v] @ V[v]) + beta1 * (Dd[v] @ U[v])
      U[v] = U[v] * (Un / torch.maximum(Ud, eps))

      sV = torch.stack(V, dim=0).sum(dim=0)
      # Updating Vm
      Vn = alpha[v] * G[v] * (D[v] @ X[v].T        @ U[v]) + alpha[v] * mu * Vs   + beta2 * (Wn[v] @ V[v]) + 2 * lamda * (sV - V[v])
      Vd = alpha[v] * G[v] * (D[v] @ V[v] @ U[v].T @ U[v]) + alpha[v] * mu * V[v] + beta2 * (Dn[v] @ V[v]) + 2 * lamda * (V[v])
      V[v] = V[v] * (Vn / torch.maximum(Vd, eps))

    # Updating Vs
    prod = [alpha[v] * V[v] for v in range(vv)]
    nom = torch.stack(prod, dim=0).sum(dim=0)
    Vs = nom/alpha.sum()

    # Updating Alphas
    T = torch.zeros(vv)
    for v in range(vv):
      T[v] = torch.norm(torch.norm(X[v] - U[v] @ V[v].T, dim=0), p=p) + mu/2 * torch.norm(V[v] - Vs) ** 2
      alpha[v] =  (-T[v]/gamma) ** (1/(gamma-1))

    # Calculating cost function
    # sVwV = torch.stack(V, dim=0).sum(dim=0)
    # for v in range(vv):
    #   err[t] += alpha[v] * torch.norm(torch.norm(X[v] - U[v] @ V[v].T, dim=0), p=p) \
    #   + mu/2 * torch.norm(V[v] - Vs) ** 2 + lamda * torch.norm(V[v] - sVwV) ** 2 \
    #   + beta1/2 * torch.trace(V[v].T @ (Dn[v] - Wn[v]) @ V[v]) + beta2/2 * torch.trace(U[v].T @ (Dd[v] - Wd[v]) @ U[v]) \
    #   + alpha[v] ** gamma

# plt.plot(err)


pred = KMeans(n_clusters=c).fit(Vs).labels_

nmi, acc, ari, f1mi, f1ma, f1we, pur = eval(Y, pred)
nmi, acc, ari, f1mi, f1ma, f1we, pur

(0.7451128689033152,
 0.821,
 0.6638963923326169,
 0.821,
 0.822582479490114,
 0.8225824794901139,
 0.821)

In [26]:
U

[tensor([[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]]),
 tensor([[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]]),
 tensor([[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]])]